In [47]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "password99"
host = "nv-desktop-services.apporto.com"
port = 30600
db = "AAC"
collection = "animals"
shelter = AnimalShelter(username, password, host, port, db, collection)


df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)
df.drop(columns=['age'], inplace=True)
df.drop(columns=['set'], inplace=True)
df.drop(columns=['a#ge'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center([
       html.A(
            html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            alt='Grazioso Salvare Logo',
            style={'width': '200px', 'height': 'auto'}
        ),
        href='https://www.snhu.edu',  
        target='_blank'  # Opens the link in a new tab
    ),
        html.Hr(),

        # Display the unique identifier "CBaxendale2023"
        html.Div(
            id='display_unique_identifier',
            children=["Dashboard created by CBaxendale2023"], 
            style={'fontSize': '24px', 'fontWeight': 'bold'}
        )
    ]),
    
    html.Hr(),
    dcc.RadioItems(
        id='filter-options',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain Rescue'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset',  # Single default value (not in a list)
        labelStyle={'display': 'block'}  # Display options vertically
    ),
    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        page_size=10,  # number of rows per page
        page_action='native',  # enable pagination
        sort_action='native',  # enable column sorting
        sort_mode='multi',  # allow sorting by multiple columns
        row_selectable='single',  # allow single row selection
        selected_rows=[0],  # select the first row by default
    ),

    html.Br(),
    html.Hr(),

    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div([
        html.Div(
            dcc.Graph(id='pie-chart'),  # Place pie chart here
            className='col s12 m6',
            style={'width': '50%', 'display': 'inline-block'}  # Set the width explicitly
        ),
        html.Div(
            dl.Map(id='map-id', style={'width': '1000px', 'height': '500px'}),
            className='col s12 m6',
            style={'display': 'inline-block'}  # Set display to inline-block
        )
    ])
])


#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-options', 'value')]  # Use 'filter-options' here
)
def update_dashboard(selected_filters):
    # Initialize filtered_data with the original data frame
    filtered_data = df
    
    # Filter the data frame based on the selected rescue type
    if 'Reset' not in selected_filters:
        # Filter based on the selected rescue type
        if 'Water Rescue' in selected_filters:
            filtered_data = df[
                (df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) &
                (df['sex_upon_outcome'] == 'Intact Female') &
                (df['age_upon_outcome_in_weeks'] >= 26) &
                (df['age_upon_outcome_in_weeks'] <= 156)
            ]
        if 'Mountain Rescue' in selected_filters:
            filtered_data = df[
                (df['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])) &
                (df['sex_upon_outcome'] == 'Intact Male') &
                (df['age_upon_outcome_in_weeks'] >= 26) &
                (df['age_upon_outcome_in_weeks'] <= 156)
            ]
        if 'Disaster Rescue' in selected_filters:
            filtered_data = df[
                (df['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])) &
                (df['sex_upon_outcome'] == 'Intact Male') &
                (df['age_upon_outcome_in_weeks'] >= 20) &
                (df['age_upon_outcome_in_weeks'] <= 300)
            ]

    return filtered_data.to_dict('records')

@app.callback(
    Output('pie-chart', "figure"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('filter-options', 'value')]
)
def update_pie_chart(viewData, selected_filter):
    if viewData is not None:
        df_view = pd.DataFrame.from_dict(viewData)

        if selected_filter == 'Water Rescue':
            filtered_df = df_view[
                (df_view['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']))
            ]
        elif selected_filter == 'Mountain Rescue':
            filtered_df = df_view[
                (df_view['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']))
            ]
        elif selected_filter == 'Disaster Rescue':
            filtered_df = df_view[
                (df_view['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']))
            ]
        else:
            # For "Reset" filter, show the top 10 breeds
            top_breeds = df_view['breed'].value_counts().head(10)
            labels = top_breeds.index.tolist()
            values = top_breeds.tolist()

            fig = px.pie(
                names=labels,
                values=values,
                title='Top 10 Breeds'
            )

            return fig

        breed_counts = filtered_df['breed'].value_counts()
        labels = breed_counts.index.tolist()
        values = breed_counts.tolist()

        fig = px.pie(
            names=labels,
            values=values,
            title=f'Percentage of Breeds for {selected_filter}'
        )

        return fig

    else:
        return go.Figure()


    
def update_styles(selected_columns):
    if selected_columns is None:
        return []  # Return an empty list if no columns are selected
    else:
        return [{
            'if': {'column_id': i},
            'background_color': '#D2F3FF'
        } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    if viewData is None:
        return
    elif index is None:
        return

    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else:
        row = index[0]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:22710/


/usr/local/anaconda/lib/python3.9/site-packages/plotly/express/_core.py:1073: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/usr/local/anaconda/lib/python3.9/site-packages/plotly/express/_core.py:1073: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

